# VisionBlock

> Fill in a module description here


In [ ]:
#| default_exp vision.VisionBlock

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import os
import sys
import random
import h5py
import torch
import torchvision
from sklearn.model_selection import train_test_split
import ujson
import torchvision.transforms as transforms
from fastcore.utils import *
from fedai.utils import *
from fedai.vision.downloader import *

In [ ]:
#| export
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#| export
class VisionBlock(torch.utils.data.Dataset):
    def __init__(self, cfg, train= True, download= True, transform=None):
        self.cfg = cfg
        self.config_path = os.path.join(self.cfg.data.dir_path, self.cfg.data.name , "config.json")
        self.train_path = os.path.join(self.cfg.data.dir_path, self.cfg.data.name , "train")
        self.test_path = os.path.join(self.cfg.data.dir_path, self.cfg.data.name, "test")
        self.train = train
        self.downloader = VisionDownloader(self.cfg, transform)
        if download:
            self.download_data()

        del self.downloader
        import gc
        gc.collect()

    def download_data(self):
        if isinstance(self.downloader.dataset_content, (list, tuple, np.ndarray)):
            train_data, test_data, stats = self.downloader.partition()
            print('saving')
            self.downloader.save_partitions(train_data, test_data, stats)
            
    def tensorify(self, data):
        X = torch.Tensor(data['x']).type(torch.float32)
        y = torch.Tensor(data['y']).type(torch.int64)
        return {'x': X, 'y': y}

    def load_single_client_data(self, idx):
        path, dir = (self.train_path, 'train') if self.train else (self.test_path, 'test')
        with h5py.File(os.path.join(path, f'{dir}_data.h5'), 'r') as hf_file:
            client_data = hf_file[f'client_{idx}']
            data = {key: client_data[key][:] for key in client_data.keys()}
            data = self.tensorify(data)
        return data
    
    def __getitem__(self, idx):
        if idx < 0: # manage minus idx
            idx = len(self) + idx
        return self.load_single_client_data(idx)
    
    def __len__(self):
        path, dir = (self.train_path, 'train') if self.train else (self.test_path, 'test')
        with h5py.File(os.path.join(path, f'{dir}_data.h5'), 'r') as hf_file:
            return len(hf_file.keys())

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()